In [1]:
import os
import requests
import time
import json
from tqdm import tqdm

## get artist ids

In [2]:
path = "parsed_chart_html"
json_files = sorted(os.listdir(path))
print(json_files)

['2005.jsonl', '2006.jsonl', '2007.jsonl', '2008.jsonl', '2009.jsonl', '2010.jsonl', '2011.jsonl', '2012.jsonl', '2013.jsonl', '2014.jsonl', '2015.jsonl', '2016.jsonl', '2017.jsonl', '2018.jsonl', '2019.jsonl', '2020.jsonl', '2021.jsonl', '2022.jsonl', '2023.jsonl']


In [3]:
artist_ids = set()
for json_file in json_files:
    #print(json_file)
    with open(os.path.join(path, json_file), "r", encoding='utf-8') as f:
        for line in tqdm(f):
            doc = json.loads(line)
            #print(doc)
            artists = doc["artists"]
            artist_id_list = [_[0] for _ in artists]
            artist_ids = artist_ids.union(artist_id_list)

    
print("num artist ids: ", len(artist_ids))

5141it [00:00, 135277.21it/s]
5182it [00:00, 110232.55it/s]
5200it [00:00, 79985.55it/s]
5200it [00:00, 81236.82it/s]
5299it [00:00, 77897.98it/s]
5192it [00:00, 56424.25it/s]
5184it [00:00, 53987.15it/s]
5193it [00:00, 44757.61it/s]
5200it [00:00, 43688.22it/s]
5200it [00:00, 41587.78it/s]
5300it [00:00, 41722.85it/s]
5198it [00:00, 38777.64it/s]
5200it [00:00, 40611.76it/s]
5200it [00:00, 32290.96it/s]
5200it [00:00, 24522.94it/s]
5300it [00:00, 23655.50it/s]
5200it [00:00, 22902.68it/s]
5200it [00:00, 22028.77it/s]
5200it [00:00, 21661.26it/s]

num artist ids:  2323


In [4]:
artist_ids = sorted(artist_ids)

## crawl artist ids

In [5]:
def requestsHtml(url):
    #
    PROXY_IP = "proxypool.daumsoft.com:7002"
    proxies = {"http": PROXY_IP, "https": PROXY_IP}
    
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
    headers = {"User-Agent": user_agent, "Host": "www.melon.com",
              "Cookie": "PCID=16847374038867242922976; PC_PCID=16847374038867242922976; POC=MP10; wcs_bt=s_f9c4bde066b:1684738153; _T_ANO=LN7+/1jYGuEk1pWeFf/8ws5/bdU0sBvkci0zB20oZjupLTvRok3l5dCviaJVCx9XAn8VrO3gzllm5G0oWjMpE5KMumb9bNDAviDSrJV7A5gp3nmb++/xZpPclNdvRFWh6VOEBUziQPGLhysNG3F3ZmFtpYuolLGVAj1qRtMG2dpQ4Tc7djgs/gDhPY0sGe77p7HCRdKzfha3wF9G5NnwWYXr0wD+9ZHxKnqHqrtTVEmEzxIbBAnnXTNl8Tm4ekHjlvmO5duev1gUgiC6KiOXYW0g9I7QldLTUnFkS9O8Yti5d+xw5Ec6DExLjyBJNNRP/N8jziBiPmn/2q36Yv3jig==",
              }
    #resp = requests.get(url, headers=headers, proxies=proxies)
    resp = requests.get(url, headers=headers) #, proxies=proxies)
    #
    return resp

In [6]:
artist_html_path = "htmls/artists"
file_names = os.listdir(artist_html_path)
alread_crawled_artist_ids = set([_.replace(".html", "") for _ in file_names])

# 이미 parse된 artist를 읽는다.
try: 
    with open("artist_infos.jsonl", "r", encoding='utf-8') as f:
        lines = f.readlines()
        docs = [json.loads(_) for _ in lines]
        parsed_artist_ids = [_["artist_num"] for _ in docs]
except:
    parsed_artist_ids = []

In [7]:
print("artist_id nums: {}".format(len(artist_ids)))
print("already crawled: {}".format(len(alread_crawled_artist_ids)))
print("already parsed: {}".format(len(parsed_artist_ids)))

artist_ids_to = [_ for _ in artist_ids if _ not in alread_crawled_artist_ids and _ not in parsed_artist_ids]
print("artist_ids to crawl: {}".format(len(artist_ids_to)))
artist_ids_to

artist_id nums: 2323
already crawled: 29
already parsed: 2553
artist_ids to crawl: 18


['2006344',
 '2114930',
 '2138414',
 '2138416',
 '2189803',
 '2243398',
 '2399727',
 '2960225',
 '2970177',
 '3059851',
 '3120571',
 '3207401',
 '3466604',
 '3478478',
 '3480275',
 '3610529',
 '773385',
 '943079']

In [8]:
for artist_id in tqdm(artist_ids_to):
    # https://www.melon.com/artist/timeline.htm?artistId=1034
    url = "https://www.melon.com/artist/detail.htm?artistId={}".format(artist_id)
    resp = requestsHtml(url)
    
    
    if "잘못된 경로로 접근" in resp.text:
        print("잘못된 경로", artist_id)
        time.sleep(2)
        continue
        
    #
    with open(os.path.join(artist_html_path, "{}.html".format(artist_id)), "w", encoding='utf-8') as of:
        print(resp.text, file=of)
        
    # 2초이상 쉬어야 끊기지 않는다.
    time.sleep(2)

100%|██████████| 18/18 [00:38<00:00,  2.13s/it]
